In [1]:
import torch

## Basic PyTorch operations

#### How to init a tensor?

In [2]:
x = torch.tensor([[1,2,3], [4,5,6]])
y = torch.tensor([[7,8,9],[10,11,12]])
f = 2 * x + y
print(f)

tensor([[ 9, 12, 15],
        [18, 21, 24]])


In [3]:
shape = [2, 3]
xzeros = torch.zeros(shape)
yzeros = torch.ones(shape)
xrnd = torch.rand(shape)
print(xzeros)
print(yzeros)
print(xrnd)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.9005, 0.6566, 0.5931],
        [0.9195, 0.3709, 0.2754]])


In [4]:
torch.manual_seed(42)
print(torch.rand([2, 3]))

tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009]])


### converting between Numpy arrays and tensors

In [6]:
import numpy as np
xnp = np.array([[1,2,3],[4,5,6]])
f2 = xnp + y  # may not working in mac.
print(f2)
print(f2.type)

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [7]:
xtensor = torch.from_numpy(xnp)
print(xtensor)
print(xtensor.type())

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.LongTensor


In [8]:
print(f.type())
fnp = f.numpy()
print(type(fnp))

torch.LongTensor
<class 'numpy.ndarray'>


In [9]:
a = np.ones(3)
t = torch.from_numpy(a)
b = t.numpy()
b[1] = 0                 # change a value in the array
print(a[1] == b[1])      # this value changes in the original array
print(t)                 # and also in the tensor - They share the same memory!!!

True
tensor([1., 0., 1.], dtype=torch.float64)


### dtype problem

In [10]:
int8np = np.ones((2,3), dtype=np.int8)
good = torch.from_numpy(int8np.astype(np.int32))
good.type()

'torch.IntTensor'

## Slicing, Indexing, Reshaping

In [13]:
print(x[0])
print(x[1][0:2])

tensor([1, 2, 3])
tensor([4, 5])
tensor([1, 2, 3, 4, 5, 6])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])


In [ ]:
print(x.view(-1))
print(x.view(3,2))
print(x.view(6,1))

In [14]:
print(x.transpose(0,1))

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [16]:
a = torch.ones(1,2,3,4)
print(a.transpose(0,3).transpose(1,2).size())
print(a.permute(3,2,1,0).size())

torch.Size([4, 3, 2, 1])
torch.Size([4, 3, 2, 1])


### inplace operators

In [17]:
print(x)
x.transpose_(1, 0)
print(x)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [ ]:
print(y)
y.add_(x*2)
print(y)

## Loading data

In [18]:
import torch
import torchvision
import torchvision.transforms as transforms

trainset = torchvision.datasets.CIFAR10(root='./data', download=True, transform=transforms.ToTensor())
trainset

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
for i in range(len(trainset)):
    print('size of image {} label {}'.format(trainset[i][0].size(), trainset[i][1]))
    if i > 2: break

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

torchimage = trainset[0][0]
npimage = torchimage.permute(1, 2, 0)
plt.imshow(npimage)

### batch and shuffle data

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(labels[0:])
print(images.size())

### Creating a custom dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import csv
import os

class ToyDataset(Dataset):
    def __init__(self, dataPath, labelsFile, transform=None):
        self.dataPath = dataPath
        self.transform = transform
        with open(os.path.join(self.dataPath, labelsFile)) as f:
            self.labels = [tuple[line] for line in csv.reader(f)]
        for i in range(len(self.labels)):
            assert os.path.isfile(dataPath + "/" + self.labels[i][0])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        imageName, imageLabel = self.labels[idx][0:]
        imagePath = os.path.join(self.dataPath, imageName)
        image = Image.open(open(imagePath, 'rb'))

        if self.transform:
            image = self.transform(image)
        return ((image, imageLabel))

### transform and concatenating datasets

In [ ]:
from torchvision import datasets
tforms = transforms.Compose([transforms.Grayscale(3), transforms.CenterCrop(300), transforms.ToTensor()])
dataFromFolders = datasets.ImageFolder(root='data/GiuseppeToys/images', transform=tforms)
folderLoader = DataLoader(dataFromFolders, batch_size=4, shuffle=True)
images, labels = iter(folderLoader).next()
print(labels)

In [ ]:
class RemoveChannel(object):
    def __init__(self, color):
        #color = 'r' 'g' or 'b' - determines which color to remove
        self.color=color

    def __call__(self, image):
        if self.color == 'r':
            image[0,:,:] = 0
        elif self.color == 'b':
            image[1,:,:] = 0
        elif self.color == 'g':
            image[2,:,:] = 0
        return(image)
cc2, cc3 = RemoveChannel('b'), RemoveChannel('g')
tforms2 = transforms.Compose([transforms.CenterCrop(500), transforms.ToTensor(), cc2])
tforms3 = transforms.Compose([transforms.CenterCrop(500), transforms.ToTensor(), cc3])
toyData2 = ToyDataset('./data/GiuseppeToys', 'labels.csv', transform=tforms2, train=True)
toyData3 = ToyDataset('./data/GiuseppeToys', 'labels.csv', transform=tforms3, train=True)
concatDataset = torch.utils.data.ConcatDataset([toyData2, toyData3])
len(concatDataset)